# Building an SVM with BOVW

## Training the SVM with SIFT features

In [1]:
#!/usr/local/bin/python2.7

import argparse as ap
# Importing library that supports user friendly commandline interfaces
import cv2
# Importing the opencv library
import numpy as np
# Importing the array operations library for python
import os
# Importing the library which supports standard systems commands
from scipy.cluster.vq import *
# Importing the library which classifies set of observations into clusters
from sklearn.preprocessing import StandardScaler
# Importing the library that supports centering and scaling vectors
from sklearn.svm import LinearSVC
# Importing the svm library
from sklearn.externals import joblib
# Importing library that supports the functionality to save the dictionary
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def imlist(path):
    """
    The function imlist returns all the names of the files in 
    the directory path supplied as argument to the function.
    """
    return [os.path.join(path, f) for f in os.listdir(path)]

def imshow(im_title, im):
    ''' This is function to display the image'''
    plt.figure()  
    plt.title(im_title)
    plt.axis("off")
    if len(im.shape) == 2:
        plt.imshow(im, cmap = "gray")
    else:
        im_display = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
        plt.imshow(im_display)
    plt.show()

def imreads(path):
    '''
    This reads all the images in a given folder and returns the results 
    '''
    images_path = imlist("/home/bikz05/Desktop/back_projection")
    images = []
    for image_path in images_path:
        images.append(cv2.imread(image_path, cv2.CV_LOAD_IMAGE_COLOR))
    return images

def show(image, name="Image"):
    '''
    Routine to display the image.
    '''
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, image)
    cv2.waitKey(0)

In [3]:
train_path = "../dataset/train/"
training_names = os.listdir(train_path)

In [4]:
# Get all the path to the images and save them in a list
# image_paths and the corresponding label in image_paths
image_paths = []
image_classes = []
class_id = 0
for training_name in training_names:
    dir = os.path.join(train_path, training_name)
    class_path = imlist(dir)
    image_paths+=class_path
    image_classes+=[class_id]*len(class_path)
    class_id+=1
# print image_classes

In [6]:
# Create feature extraction and keypoint detector objects
fea_det = cv2.xfeatures2d.SIFT_create()
# des_ext = cv2.DescriptorExtractor_create("SIFT")
surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create(nfeatures=1500)
# brief = cv2.xfeatures2d.brief()
# List where all the descriptors are stored
des_list = []

error_count = 0
for image_path in image_paths:
    im = cv2.imread(image_path)
    try:
        (kps, des) = fea_det.detectAndCompute(im, None)
#         print des.shape, des2.shape, des3.shape
    except:
        print image_path
    if des is not None:
        des_list.append((image_path, des))
    else:
        print image_path
print "done"
# Stack all the descriptors vertically in a numpy array
descriptors = des_list[0][1]
for image_path, descriptor in des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))


done


In [7]:
# Perform k-means clustering
k = 100
voc, variance = kmeans(descriptors, k, 1)

# Calculate the histogram of features
im_features = np.zeros((len(image_paths), k), "float32")
for i in xrange(len(image_paths)):
    words, distance = vq(des_list[i][1],voc)
    for w in words:
        im_features[i][w] += 1 # Caluculating the histogram of features

In [8]:
# Perform Tf-Idf vectorization
nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
# Calculating the number of occurrences
idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')
# Giving weight to one that occurs more frequently

# Scaling the words
stdSlr = StandardScaler().fit(im_features)
im_features = stdSlr.transform(im_features)  # Scaling the visual words for better Prediction

# Train the Linear SVM
clf = LinearSVC()
clf.fit(im_features, np.array(image_classes))

# Save the SVM
joblib.dump((clf, training_names, stdSlr, k, voc), "bof_SIFT.pkl", compress=3)

['bof_SIFT.pkl']

## Testing the SVM 

In [9]:
import argparse as ap
# Importing library that supports user friendly commandline interfaces
import cv2
# Importing the opencv library
import numpy as np
# Importing the array operations library for python
import os
# Importing the library which supports standard systems commands
from sklearn.svm import LinearSVC
# Importing the library that supports linear support vector machine
from sklearn.externals import joblib
from scipy.cluster.vq import *
# Importing the library which classifies set of observations into clusters
from sklearn.preprocessing import StandardScaler
# Importing the library that supports centering and scaling vectors

In [10]:
# Load the classifier, class names, scaler, number of clusters and vocabulary
clf, classes_names, stdSlr, k, voc = joblib.load("bof_SIFT.pkl")
test_path = "../dataset/test/"

In [11]:
# Get the path of the testing image(s) and store them in a list
image_paths = []
image_classes = []
class_id = 0
testing_names = os.listdir(test_path)
for testing_name in testing_names:
        dir = os.path.join(test_path, testing_name)
        class_path = imlist(dir)
        image_paths+=class_path
        image_classes+=[class_id]*len(class_path)
        class_id+=1

In [12]:
# Create feature extraction and keypoint detector objects
fea_det = cv2.xfeatures2d.SIFT_create()
surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create(nfeatures=1500)

# des_ext = cv2.DescriptorExtractor_create("SIFT")

# List where all the descriptors are stored
des_list = []

for image_path in image_paths:
    im = cv2.imread(image_path)
    if im is None:
        print "No such file {}\nCheck if the file exists".format(image_path)
        exit()
    (kpts, des) = fea_det.detectAndCompute(im, None)
    # kpts = fea_det.detect(im)  # Computing the key points of test image
    # kpts, des = des_ext.compute(im, kpts)  # Computing the descriptors of the test image
    if des is not None:
        des_list.append((image_path, des))  # Appending the descriptors to a single list
    else:
        print image_path

In [13]:
# Stack all the descriptors vertically in a numpy array
descriptors = des_list[0][1]
for image_path, descriptor in des_list[0:]:
    descriptors = np.vstack((descriptors, descriptor))  # Stacking the descriptors in to a numpy array

# # Calculating the histogram of features
test_features = np.zeros((len(image_paths), k), "float32")
for i in xrange(len(image_paths)):
    words, distance = vq(des_list[i][1],voc)
    for w in words:
        test_features[i][w] += 1
        # Calculating the histogram of features

# Perform Tf-Idf vectorization
nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
# Getting the number of occurrences of each word
idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')
# Assigning weight to one that is occurring more frequently

# Scale the features
test_features = stdSlr.transform(test_features)

# Perform the predictions
predictions =  [classes_names[i] for i in clf.predict(test_features)]


In [14]:
wrong_count_0 = 0
wrong_count_1 = 0
n1 = len(os.listdir("../dataset/test/birds/"))
n0 = len(os.listdir("../dataset/test/drones/"))
pred_id = []
actual = []
for i in xrange(len(image_paths)):
    if predictions[i] == "birds":
        pred_id.append("birds")
    else:
        pred_id.append("drones")
    if image_classes[i] == "birds":
        actual.append("birds")
    else:
        actual.append("drones")
#     if predictions[i] == "birds" and image_classes[i] != 1:
#         # print predictions[i], image_classes[i]
#         wrong_count_1 +=1
#     if predictions[i] == "drones" and image_classes[i] != 0:
#         # print predictions[i], image_classes[i]
#         wrong_count_0 +=1
    if image_classes[i] == 0 and predictions[i] == "birds":
        wrong_count_0 +=1
    if image_classes[i] == 1 and predictions[i] == "drones":
        wrong_count_1 +=1
print "Accuracy:" , float(len(image_paths) - wrong_count_0 - wrong_count_1)/len(image_paths)

Accuracy: 0.726038338658


In [15]:
conf_matrix = [[n0-wrong_count_0, wrong_count_0], [wrong_count_1, n1-wrong_count_1]]
print "Confusion Matrix: \n", conf_matrix

Confusion Matrix: 
[[318, 108], [235, 591]]
